In [30]:
import wandb
wandb.init(project="pytorch-ignite-example")

W&B Run: https://app.wandb.ai/evs/pytorch-ignite-example/runs/csabzg3f

In [37]:
import torch
from torch import nn
from torch.optim import SGD
from torch.utils.data import DataLoader
import torch.nn.functional as F

from torchvision.transforms import Compose, ToTensor, Normalize
from torchvision.datasets import MNIST


from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
        
from ignite.metrics import Accuracy, Loss

from tqdm import tqdm


In [44]:
class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_features, num_samples):
        super(VariationalAutoencoder, self).__init__()
        
        self.latent_features = latent_features
        self.num_samples = num_samples

        # We encode the data onto the latent space using two linear layers
        self.encoder = nn.Sequential(
            nn.Linear(in_features=num_features, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=64),
            nn.ReLU(),
            # A Gaussian is fully characterised by its mean and variance
            nn.Linear(in_features=64, out_features=2*self.latent_features) # <- note the 2*latent_features
        )
        
        # The latent code must be decoded into the original image
        self.decoder = nn.Sequential(
            nn.Linear(in_features=self.latent_features, out_features=64),
            nn.LeakyReLU(),
            nn.Linear(in_features=64, out_features=128),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=256),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=num_features)
        )
        

    def forward(self, x): 
        outputs = {}
        
        # Split encoder outputs into a mean and variance vector
        mu, log_var = torch.chunk(self.encoder(x), 2, dim=-1)
        
        # :- Reparametrisation trick
        # a sample from N(mu, sigma) is mu + sigma * epsilon
        # where epsilon ~ N(0, 1)
                
        # Don't propagate gradients through randomness
        with torch.no_grad():
            batch_size = mu.size(0)
            epsilon = torch.randn(batch_size, self.num_samples, self.latent_features)
            
            if cuda:
                epsilon = epsilon.cuda()
        
        sigma = torch.exp(log_var/2)
        
        # We will need to unsqueeze to turn
        # (batch_size, latent_dim) -> (batch_size, 1, latent_dim)
        z = mu.unsqueeze(1) + epsilon * sigma.unsqueeze(1)        
        
        # Run through decoder
        x = self.decoder(z)
        
        # The original digits are on the scale [0, 1]
        x = torch.sigmoid(x)
        
        # Mean over samples
        x_hat = torch.mean(x, dim=1)
        
        outputs["x_hat"] = x_hat
        outputs["z"] = z
        outputs["mu"] = mu
        outputs["log_var"] = log_var
        
        return outputs


In [42]:
def get_data_loaders(train_batch_size, val_batch_size):
    data_transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])

    train_loader = DataLoader(MNIST(download=True, root=".", transform=data_transform, train=True),
                              batch_size=train_batch_size, shuffle=True)

    val_loader = DataLoader(MNIST(download=False, root=".", transform=data_transform, train=False),
                            batch_size=val_batch_size, shuffle=False)
    return train_loader, val_loader


def run(train_batch_size, val_batch_size, epochs, lr, momentum, log_interval):
    train_loader, val_loader = get_data_loaders(train_batch_size, val_batch_size)
    model = Net()
    wandb.watch(model)
    device = 'cpu'

    if torch.cuda.is_available():
        device = 'cuda'

    optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
    trainer = create_supervised_trainer(model, optimizer, F.nll_loss, device=device)
    evaluator = create_supervised_evaluator(model,
                                            metrics={'accuracy': Accuracy(),
                                                     'nll': Loss(F.nll_loss)},
                                            device=device)

    desc = "ITERATION - loss: {:.2f}"
    pbar = tqdm(
        initial=0, leave=False, total=len(train_loader),
        desc=desc.format(0)
    )

    @trainer.on(Events.ITERATION_COMPLETED(every=log_interval))
    def log_training_loss(engine):
        pbar.desc = desc.format(engine.state.output)
        pbar.update(log_interval)
        wandb.log({"train loss": engine.state.output})

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_training_results(engine):
        pbar.refresh()
        evaluator.run(train_loader)
        metrics = evaluator.state.metrics
        avg_accuracy = metrics['accuracy']
        avg_nll = metrics['nll']
        tqdm.write(
            "Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
            .format(engine.state.epoch, avg_accuracy, avg_nll)
        )

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(engine):
        evaluator.run(val_loader)
        metrics = evaluator.state.metrics
        avg_accuracy = metrics['accuracy']
        avg_nll = metrics['nll']
        tqdm.write(
            "Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
            .format(engine.state.epoch, avg_accuracy, avg_nll))

        pbar.n = pbar.last_print_n = 0
        wandb.log({"validation loss": avg_nll})
        wandb.log({"validation accuracy": avg_accuracy})

    trainer.run(train_loader, max_epochs=epochs)
    pbar.close()

In [43]:
# Train Model

hyperparameter_defaults = dict(
    batch_size = 256,
    val_batch_size = 100,
    epochs = 10,
    lr = 0.001,
    momentum = 0.3,
    log_interval = 10,
)


# Get metrics in Weights & Biases
wandb.init(config=hyperparameter_defaults, project="pytorch-ignite-example")
config = wandb.config
run(config.batch_size, config.val_batch_size, config.epochs, config.lr, config.momentum, config.log_interval)



ITERATION - loss: 0.00:   0%|          | 0/235 [00:00<?, ?it/s]

ITERATION - loss: 2.32:   4%|▍         | 10/235 [00:00<00:16, 13.24it/s]

ITERATION - loss: 2.31:   9%|▊         | 20/235 [00:01<00:16, 13.18it/s]

ITERATION - loss: 2.31:  13%|█▎        | 30/235 [00:02<00:16, 12.44it/s]

ITERATION - loss: 2.29:  17%|█▋        | 40/235 [00:03<00:15, 12.43it/s]

ITERATION - loss: 2.32:  21%|██▏       | 50/235 [00:03<00:14, 13.11it/s]

ITERATION - loss: 2.32:  26%|██▌       | 60/235 [00:04<00:12, 13.60it/s]

ITERATION - loss: 2.33:  30%|██▉       | 70/235 [00:05<00:12, 13.10it/s]

ITERATION - loss: 2.32:  34%|███▍      | 80/235 [00:06<00:13, 11.78it/s]

ITERATION - loss: 2.29:  38%|███▊      | 90/235 [00:07<00:12, 11.59it/s]

ITERATION - loss: 2.29:  43%|████▎     | 100/235 [00:08<00:12, 11.03it/s]

ITERATION - loss: 2.32:  47%|████▋     | 110/235 [00:09<00:11, 10.92it/s]

ITERATION - loss: 2.30:  51%|█████     | 120/235 [00:10<00:10, 11.01it/s]

ITERATION - loss: 2.31:  55%|█████▌    | 1

Training Results - Epoch: 1  Avg accuracy: 0.17 Avg loss: 2.28


                                                                       
                                                                    

ITERATION - loss: 2.30:   0%|          | 0/235 [02:34<00:15, 15.30it/s]

ITERATION - loss: 2.30:  98%|█████████▊| 230/235 [00:31<00:00, 14.57it/s]

Validation Results - Epoch: 1  Avg accuracy: 0.16 Avg loss: 2.28




ITERATION - loss: 2.29:   4%|▍         | 10/235 [00:32<01:45,  2.13it/s] 

ITERATION - loss: 2.29:   9%|▊         | 20/235 [00:32<01:15,  2.85it/s]

ITERATION - loss: 2.28:  13%|█▎        | 30/235 [00:33<00:54,  3.75it/s]

ITERATION - loss: 2.26:  17%|█▋        | 40/235 [00:34<00:40,  4.82it/s]

ITERATION - loss: 2.28:  21%|██▏       | 50/235 [00:34<00:30,  5.98it/s]

ITERATION - loss: 2.28:  26%|██▌       | 60/235 [00:35<00:24,  7.23it/s]

ITERATION - loss: 2.29:  30%|██▉       | 70/235 [00:36<00:19,  8.38it/s]

ITERATION - loss: 2.27:  34%|███▍      | 80/235 [00:37<00:16,  9.63it/s]

ITERATION - loss: 2.30:  38%|███▊      | 90/235 [00:37<00:13, 10.80it/s]

ITERATION - loss: 2.27:  43%|████▎     | 100/235 [00:38<00:11, 11.37it/s]

ITERATION - loss: 2.27:  47%|████▋     | 110/235 [00:39<00:10, 12.15it/s]

ITERATION - loss: 2.29:  51%|█████     | 120/235 [00:39<00:09, 12.66it/s]

ITERATION - loss: 2.28:  55%|█████▌    | 130/235 [00:40<00:07, 13.14it/s]

ITERATION - loss: 2.27:  60%|██

...

Validation Results - Epoch: 9  Avg accuracy: 0.80 Avg loss: 0.93




ITERATION - loss: 1.34:   4%|▍         | 10/235 [04:04<01:29,  2.52it/s] 

ITERATION - loss: 1.32:   9%|▊         | 20/235 [04:05<01:03,  3.37it/s]

ITERATION - loss: 1.31:  13%|█▎        | 30/235 [04:05<00:46,  4.41it/s]

ITERATION - loss: 1.35:  17%|█▋        | 40/235 [04:06<00:34,  5.60it/s]

ITERATION - loss: 1.31:  21%|██▏       | 50/235 [04:07<00:26,  6.94it/s]

ITERATION - loss: 1.17:  26%|██▌       | 60/235 [04:07<00:20,  8.34it/s]

ITERATION - loss: 1.25:  30%|██▉       | 70/235 [04:08<00:16,  9.71it/s]

ITERATION - loss: 1.30:  34%|███▍      | 80/235 [04:09<00:14, 10.97it/s]

ITERATION - loss: 1.20:  38%|███▊      | 90/235 [04:09<00:11, 12.08it/s]

ITERATION - loss: 1.20:  43%|████▎     | 100/235 [04:10<00:10, 12.98it/s]

ITERATION - loss: 1.23:  47%|████▋     | 110/235 [04:10<00:09, 13.57it/s]

ITERATION - loss: 1.18:  51%|█████     | 120/235 [04:11<00:08, 14.06it/s]

ITERATION - loss: 1.27:  55%|█████▌    | 130/235 [04:12<00:07, 14.42it/s]

ITERATION - loss: 1.24:  60%|██

Training Results - Epoch: 10  Avg accuracy: 0.81 Avg loss: 0.81


                                                                       
                                                                    

ITERATION - loss: 2.30:   0%|          | 0/235 [06:32<00:15, 15.30it/s]

ITERATION - loss: 1.11: 240it [04:30, 15.74it/s]

                                                

Validation Results - Epoch: 10  Avg accuracy: 0.82 Avg loss: 0.79
